Marcos Gonzalez  
Homework #15    
4/6/21

# Homework 15

This homework uses the Green taxi data from Homework 2 and 14.  The 2020 green taxi trip data from NYC Open Data was filtered to only be trips with a pickup date and time on Feb. 4, 2020. 

The `RatecodeID` column is a qualitative column with the following categories:
* 1 = Standard rate
* 2 = JFK (flat rate from JFK to Manhattan)
* 3 = Newark
* 4 = Nassau or Westchester
* 5 = Negotiated fare
* 6 = Group ride

Filtered trip data URL: [https://raw.githubusercontent.com/megan-owen/MAT328-Techniques_in_Data_Science/main/data/Feb4_2020_Green_Taxi_Trip_Data.csv](https://raw.githubusercontent.com/megan-owen/MAT328-Techniques_in_Data_Science/main/data/Feb4_2020_Green_Taxi_Trip_Data.csv)

We will try to predict which trips involve tolls using the `RatecodeId`, `passenger_count`, `trip_distance`, and `total_amount` columns.

**Answer all questions in the homework, including ones that require written answers.**  For questions related to Homework 14, you can use the Homework 14 solutions posted on Blackboard if you did not complete Homework 14.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import statsmodels.formula.api as smf
from scipy.special import expit
from scipy.stats import logistic

%matplotlib inline

### Question 1: Loading and cleaning the data

1a) Load the data into Python.

In [2]:
taxi = pd.read_csv("https://raw.githubusercontent.com/megan-owen/MAT328-Techniques_in_Data_Science/main/data/Feb4_2020_Green_Taxi_Trip_Data.csv")
taxi.head()

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge
0,2.0,02/04/2020 12:00:07 AM,02/04/2020 12:08:01 AM,N,1.0,42,152,1.0,1.21,7.00,0.50,0.5,0.0,0.0,NaN,0.3,8.3,1.0,1.0,0.0
1,2.0,02/04/2020 12:00:20 AM,02/04/2020 12:13:44 AM,N,1.0,25,188,1.0,3.28,12.50,0.50,0.5,0.0,0.0,NaN,0.3,13.8,2.0,1.0,0.0
2,2.0,02/04/2020 12:00:42 AM,02/04/2020 12:03:29 AM,N,1.0,166,152,1.0,0.68,4.00,0.50,0.5,0.0,0.0,NaN,0.3,5.3,2.0,1.0,0.0
3,2.0,02/04/2020 12:01:00 AM,02/04/2020 12:13:15 AM,N,1.0,82,173,1.0,1.89,9.50,0.50,0.5,1.5,0.0,NaN,0.3,12.3,1.0,1.0,0.0
4,NaN,02/04/2020 12:01:00 AM,02/04/2020 12:31:00 AM,NaN,NaN,136,90,NaN,12.80,51.55,2.75,0.0,0.0,0.0,NaN,0.3,54.6,NaN,NaN,NaN


Clean the data as follows:

1b) Create a new column call `paid_toll` that contains a 1 if the value in the `tolls_amount` column is greater than 0 and a 0 otherwise.  Hint:  See Lab 1, Section 6.

In [4]:
def is_positive(x):
  if x > 0:
    return 1
  else:
    return 0

1c) Create a new DataFrame (or change your current DataFrame) that only contains the columns `RatecodeID`, `passenger_count`,`trip_distance`, `total_amount`, and `paid_toll`

In [5]:
taxi["paid_toll"] = taxi["tolls_amount"].apply(is_positive)
taxi2 = taxi[["RatecodeID","passenger_count","trip_distance","total_amount","paid_toll"]]
taxi2

,RatecodeID,passenger_count,trip_distance,total_amount,paid_toll
0,1.0,1.0,1.21,8.30,0
1,1.0,1.0,3.28,13.80,0
2,1.0,1.0,0.68,5.30,0
3,1.0,1.0,1.89,12.30,0
4,NaN,NaN,12.80,54.60,0
...,...,...,...,...,...
13976,1.0,1.0,0.64,5.80,0
13977,1.0,1.0,1.77,10.30,0
13978,1.0,1.0,1.80,11.44,0
13979,1.0,1.0,0.69,5.80,0


1d) Do any of the rows in the DataFrame created in question 1c have missing values?  If so, drop all rows with at least one NaN values.   How many rows did you drop?

In [10]:
missing_rate = taxi2["RatecodeID"].isna()
taxi_rate = taxi2[~ missing_rate]
missing_pass = taxi_rate["passenger_count"].isna()
taxi_pass = taxi_rate[~ missing_pass]
missing_dis = taxi_pass["trip_distance"].isna()
taxi_dis = taxi_pass[~ missing_dis]
missing_amount = taxi_dis["total_amount"].isna()
taxi3 = taxi_dis[~ missing_amount]
taxi3

,RatecodeID,passenger_count,trip_distance,total_amount,paid_toll
0,1.0,1.0,1.21,8.30,0
1,1.0,1.0,3.28,13.80,0
2,1.0,1.0,0.68,5.30,0
3,1.0,1.0,1.89,12.30,0
5,1.0,2.0,4.57,17.30,0
...,...,...,...,...,...
13976,1.0,1.0,0.64,5.80,0
13977,1.0,1.0,1.77,10.30,0
13978,1.0,1.0,1.80,11.44,0
13979,1.0,1.0,0.69,5.80,0


3447 rows were dropped

1d) Change the type of the `RatecodeID` column to `int` to make referencing the dummy variables easier, and then replace the `RatecodeID` column with dummy variables for it  Hint: See lab 4, Section 5 for changing the type of the column.


In [12]:
taxi3["RatecodeID"] = taxi3["RatecodeID"].astype(int)

In [13]:
taxi3.dtypes

RatecodeID           int64
passenger_count    float64
trip_distance      float64
total_amount       float64
paid_toll            int64
dtype: object

In [14]:
taxi4 = pd.get_dummies(taxi3, columns = ["RatecodeID"], drop_first = True)
taxi4

,passenger_count,trip_distance,total_amount,paid_toll,RatecodeID_2,RatecodeID_3,RatecodeID_4,RatecodeID_5
0,1.0,1.21,8.30,0,0,0,0,0
1,1.0,3.28,13.80,0,0,0,0,0
2,1.0,0.68,5.30,0,0,0,0,0
3,1.0,1.89,12.30,0,0,0,0,0
5,2.0,4.57,17.30,0,0,0,0,0
...,...,...,...,...,...,...,...,...
13976,1.0,0.64,5.80,0,0,0,0,0
13977,1.0,1.77,10.30,0,0,0,0,0
13978,1.0,1.80,11.44,0,0,0,0,0
13979,1.0,0.69,5.80,0,0,0,0,0


### Question 2: Logistic regression

2a) Create a logistic model where `paid_toll` is the dependent variable, and all other columns in your final, cleaned DataFrame from Question 1 are the independent variables.

In [15]:
logit_model = smf.logit('paid_toll ~ passenger_count + trip_distance + total_amount +\
    RatecodeID_2 + RatecodeID_3 + RatecodeID_4 + RatecodeID_5', \
                        data = taxi4).fit()
logit_model.summary()

Optimization terminated successfully.
         Current function value: 0.068240
         Iterations 9


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:              paid_toll   No. Observations:                10534
Model:                          Logit   Df Residuals:                    10526
Method:                           MLE   Df Model:                            7
Date:                Tue, 06 Apr 2021   Pseudo R-squ.:                  0.4086
Time:                        17:29:42   Log-Likelihood:                -718.84
converged:                       True   LL-Null:                       -1215.5
Covariance Type:            nonrobust   LLR p-value:                3.261e-210
===================================================================================
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept          -6.8267      0.217    -31.405      0.000      -7.253      -6.401
passenger_count     0.0860      0.076      1.133      0.257      -0.063       0.235
trip_distance      -0.0258      0.021     -1.223      0.221      -0.067       0.016
total_amount        0.1253      0.007     16.886      0.000       0.111       0.140
RatecodeID_2       -0.4159      0.496     -0.839      0.402      -1.388       0.556
RatecodeID_3       -1.0865      9.380     -0.116      0.908     -19.471      17.298
RatecodeID_4       -3.8788      1.381     -2.810      0.005      -6.585      -1.173
RatecodeID_5       -1.1538      0.535     -2.158      0.031      -2.202      -0.106
===================================================================================
"""

2b) Recall that the p-value for each variable (in the column P > |z|) is the probability that we would get the coefficient in the `coef` column or one with a greater magnitude, when that coefficient is really 0 (meaning that variable does not affect the dependent variable).  Make a new logistic model with only those variables with a p-value less than 0.05, and compute the summary.

In [16]:
logit_model2 = smf.logit('paid_toll ~ total_amount +\
    RatecodeID_4 + RatecodeID_5', \
                        data = taxi4).fit()
logit_model2.summary()

Optimization terminated successfully.
         Current function value: 0.068378
         Iterations 9


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:              paid_toll   No. Observations:                10534
Model:                          Logit   Df Residuals:                    10530
Method:                           MLE   Df Model:                            3
Date:                Tue, 06 Apr 2021   Pseudo R-squ.:                  0.4074
Time:                        17:32:34   Log-Likelihood:                -720.29
converged:                       True   LL-Null:                       -1215.5
Covariance Type:            nonrobust   LLR p-value:                2.104e-214
================================================================================
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept       -6.6329      0.173    -38.299      0.000      -6.972      -6.293
total_amount     0.1173      0.005     25.616      0.000       0.108       0.126
RatecodeID_4    -3.7002      1.383     -2.676      0.007      -6.410      -0.990
RatecodeID_5    -1.0520      0.523     -2.010      0.044      -2.078      -0.026
================================================================================
"""

2c)  What is the Pseudo R-squared value for your new logistic model?  What does it suggest about the fit of the model?  How does this compare to the pseudo r-squared value for all your first logistic regression model with all the variables?  How does it compare to the logistic regression model from Homework 14 that only used `trip_distance` as the independent variable?

Pseudo R-squared: 0.4074  
Compared to the first logistic regression model there isn't much change between both of them. But both are a improvement over the logistic regression model from homework 14.

2d)  What is the equation of your logistic regression model?  The equation below is written in Latex, which is a system for typing math equations.  Double click on this cell, and replace the part of the equation that says "reg" to finish the formula.  "run" this cell (as you would a code cell) when you are finished.

$$y = \frac{1}{1 + e^{-(-6.6329+0.1173x_1-3.7002x_2-1.0520x_3)}}$$

2f) Compute the confusion matrix for your second logistic regression model.  What is the accuracy, sensitivity, specificity, and precision of your model?  What does mean the strengths and weaknesses of the model are?

In [17]:
logit_model2.pred_table()

array([[10232.,    43.],
       [  191.,    68.]])

In [21]:
# accuracy
(10232 + 43)/len(taxi4)

0.9754129485475603

In [18]:
# sensitivity
68/(191+68)

0.2625482625482625

In [19]:
# specificity
10232/(10232+43)

0.9958150851581509

In [22]:
# precision
10232/(10232 + 191)

0.9816751415139595

The sensitivity seems to be the weakest thing in this model while the rest seem to be pretty good.